# MAT281 - Laboratorios N°07

<a id='p1'></a>
## I.- Problema 01


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Anscombe.svg/1200px-Anscombe.svg.png" width="360" height="360" align="center"/>



El **cuarteto de Anscombe** comprende cuatro conjuntos de datos que tienen las mismas propiedades estadísticas, pero que evidentemente son distintas al inspeccionar sus gráficos respectivos.

Cada conjunto consiste de once puntos (x, y) y fueron construidos por el estadístico F. J. Anscombe. El cuarteto es una demostración de la importancia de mirar gráficamente un conjunto de datos antes de analizarlos.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
# cargar datos
df = pd.read_csv(os.path.join("data","anscombe.csv"), sep=",")
df.head()

Basado en la información presentada responda las siguientes preguntas:

1. Gráfique mediante un gráfico tipo **scatter** cada grupo. A simple vista, ¿ los grupos son muy distintos entre si?.
2. Realice un resumen de las medidas estadísticas más significativas ocuapando el comando **describe** para cada grupo. Interprete.
3. Realice un ajuste lineal para cada grupo. Además, grafique los resultados de la regresión lineal para cada grupo. Interprete.
4. Calcule los resultados de las métricas para cada grupo. Interprete.
5. Es claro que el ajuste lineal para algunos grupos no es el correcto. Existen varias formas de solucionar este problema (eliminar outliers, otros modelos, etc.). Identifique una estrategia para que el modelo de regresión lineal ajuste de mejor manera e implemente otros modelos en los casos que encuentre necesario.  

1. **scatterplot**

In [ ]:
#1
str = ['Grupo_1', 'Grupo_2', 'Grupo_3','Grupo_4']
grupos = [df.groupby('grupo').get_group(g) for g in str] #dataframes de cada grupo
map = lambda n: [int(np.floor(n/2)),int(np.mod(n,2))] #funcion que mapea 0->(0,0) 1->(0,1) 2->(1,0) 3->(1,1)
index = [map(i) for i in range(0,4)] #indices para graficar con un solo loop

In [ ]:
fig,ejes = plt.subplots(2,2,figsize=(15,10))
for i in range(0,4):
    sns.scatterplot(
        x = 'x',
        y = 'y',
        data = grupos[i],
        ax = ejes[index[i][0]][index[i][1]],
        s=150
    )
    ejes[index[i][0]][index[i][1]].set_title(f"Grupo {i+1}")

Revisando las dispersiones de los cuatro grupos, no se parecen entre sí en cuanto a la distribución aparente de los datos.

2. **describe**

In [ ]:
#2
stats = [display(grupos[k].describe().style.set_caption(f'Grupo {k+1}')) for k in range(0,4)]

Se observa que los cuatro grupos presentan medias y varianzas muestrales similares entre si, más revisando los percentiles, mínimos y máximos mostrados por **describe()** los datos se distinguen en como estan distribuidos, tal y como se observó visualmente en (1).


3. **regresion lineal**

In [ ]:
#3
#sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split

metricas = []
for i in range(0,4):
    X = grupos[i][['x']]
    y = grupos[i]['y']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    from sklearn.linear_model import LinearRegression
    model_rl = LinearRegression()
    model_rl.fit(X_train, y_train)
    
    #lista de coef B para cada X
    beta_0 = round(model_rl.intercept_, 2)
    beta_1 = round(model_rl.coef_[0],2)
    
    func = f"f(x) = {beta_0} ´{beta_1}x"
    
    #predicciones
    Y_predict = model_rl.predict(X_test)
    
    #graficos con seaborn
    beta_0 = model_rl.intercept_
    beta_1 = model_rl.coef_[0]
    
    x_range = np.arange(0,20,0.1)
    
    df_plot = pd.DataFrame({'x':x_range,
                           'y_true':[beta_0 + beta_1*n for n in x_range]})
    
    dfa = pd.DataFrame({'x':X['x'],
                       'y_true':y})
    
    fig, ax = plt.subplots(figsize=(10, 8.5))
    
    aa = sns.scatterplot(x= 'x', y='y_true', data=dfa, ax=ax,s=150)
    aa = sns.lineplot(x='x', y='y_true', data=df_plot,ax=ax,color='red')
    plt.title(f'Regresión Lineal Grupo {i+1}: {func}', size = 15)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    
    #se guardan métricas para imprimir en pregunta 4
    from metrics_regression import *
    from sklearn.metrics import r2_score
    df_temp = pd.DataFrame(
        {
            'y':y_test,
            'yhat':model_rl.predict(X_test)
        }
    )
    
    df_metrics = summary_metrics(df_temp)
    df_metrics['r2'] = round(r2_score(y_test,model_rl.predict(X_test)),4)
    metricas.append(df_metrics)
    
    

De los scatterplot, se observa que la regresión lineal hecha para Grupo 1, podría modelar correctamente los datos, para el resto de los grupos habría que utilizar otro modelo o normalizar datos o realizar otro tipo de técnica.

4. **metricas**

In [ ]:
#4
arr = [print(f'\nMetricas para el regresor x para Grupo {i+1}:\n{metricas[i]}') for i in range(0,4)]

La única regresión relativamente aceptable sería la realizada en Grupo 1, ya que presenta bajos errores porcentuales y coeficiente $R^2$ cercano a 1. Las demás presentan altos errores porcentuales y $R^2$ negativos, lo que descarta que estén siendo modelados correctamente.

5. **modelar apropiadamente**

*Grupo 3:* Eliminar datos outliers de la muestra y realizar un nuevo ajuste lineal.

*Grupo 4:* Eliminando el único outlier, se puede ajustar con la recta $x = 8$ ya que todas las coordenadas $x$ de los datos restantes son iguales.

*Grupo 2:* Se podría ajustar mediante un modelo logarítmico.

Se realizarán los ajustes para los Grupos 3 y 4.

In [ ]:
#5
#para grupo 3 eliminar el outlier que se puede ver en (2) que es el dato con valor y máximo
group_temp = grupos[2]
group_temp = group_temp[group_temp.y != group_temp.y.max()].reset_index(drop=True)
display(group_temp)

X = group_temp[['x']]
y = group_temp['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LinearRegression
model_rl = LinearRegression()
model_rl.fit(X_train, y_train)

# lista de coeficientes B para cada X
beta_0 = round(model_rl.intercept_,2)
beta_1 = round(model_rl.coef_[0],2)

func = f"f(x) = {beta_0} + {beta_1}x"

# predicciones
Y_predict = model_rl.predict(X_test)

# graficos con seaborn
beta_0 = model_rl.intercept_
beta_1 = model_rl.coef_[0]

x_range = np.arange(0,18,0.1)

df_plot = pd.DataFrame({'x':x_range,
                        'y_true':[beta_0 + beta_1*n for n in x_range]})

dfa = pd.DataFrame({'x':X['x'], 
                    'y_true':y})

fig, ax = plt.subplots(figsize=(10, 8.5))

aa = sns.scatterplot(x='x', y='y_true', data=dfa, ax=ax, s=150)
aa = sns.lineplot(x='x', y='y_true', data=df_plot,ax=ax,color="red")
plt.title(f'Regresión Lineal Grupo {i+1}: {func}', size=15)
plt.ylabel('y')
plt.show()

# metricas
from metrics_regression import *
from sklearn.metrics import r2_score
df_temp = pd.DataFrame(
    {
        'y':y_test,
        'yhat': model_rl.predict(X_test)
        }
)

df_metrics = summary_metrics(df_temp)
df_metrics['r2'] =  round(r2_score(y_test, model_rl.predict(X_test)),4)
print(f'\nMetricas para el regresor x para Grupo 3:\n{df_metrics}')

In [ ]:
#para grupo 4 eliminar el outlier que se puede ver en (2) que es el dato con valor "x" e "y" máximo
group_temp = grupos[3]
group_temp = group_temp[group_temp.y != group_temp.y.max()].reset_index(drop=True)
display(group_temp)


fig, ax = plt.subplots(figsize=(10, 8.5))

aa = sns.scatterplot(x='x', y='y', data=group_temp, ax=ax, s=150,color="red")
plt.axvline(x=8)
plt.title(f'Regresión Lineal Grupo 4: x = 8', size=15)
plt.show()